In [ ]:
# T-test, U-test
import pandas as pd
import numpy as np
import warnings
from scipy.stats import ttest_ind, mannwhitneyu
from pitest_log_parser import project_list, round_number, seed_list, mutant_choice, test_choice
TIMED_OUT = 'TIMED_OUT'
random_mutant = True
random_test = False
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'
seed_num = len(seed_list)
warnings.filterwarnings('ignore', category=RuntimeWarning, message='Precision loss occurred in moment calculation.')


def get_mutant_runtimeList_dict(seed):
    temp_df = pd.read_csv(f'analyzed-data/{choice}/{seed}_{project}.csv')
    temp_dict = {}
    for index, row in temp_df.iterrows():
        temp_dict[row['mutant']] = [row[f'round-{cnt}'] for cnt in range(round_number)]
    return temp_dict


def handle_timedOut_noise(array):
    if all(x == TIMED_OUT for x in array) or any(x is np.nan for x in array):
        return False
    fill_value = np.mean([float(x) for x in array if x != TIMED_OUT])
    temp_array = []
    for x in array:
        if x == TIMED_OUT:
            temp_array.append(fill_value)
        else:
            temp_array.append(float(x))
    return temp_array


for project in project_list:
    for i in range(seed_num):
        seed_i = seed_list[i]
        i_dict = get_mutant_runtimeList_dict(seed_i)
        j = i + 1
        while j < seed_num:
            df = pd.DataFrame(None, columns=['mutant', 'T-test', 'U-test'])
            seed_j = seed_list[j]
            j_dict = get_mutant_runtimeList_dict(seed_j)
            pair = (seed_i, seed_j)
            for mutant, i_runtime_list in i_dict.items():
                if not mutant in j_dict:
                    continue
                j_runtime_list = j_dict[mutant]
                i_array = handle_timedOut_noise(i_runtime_list)
                j_array = handle_timedOut_noise(j_runtime_list)
                if not i_array or not j_array:
                    continue
                t_stat, t_p_value = ttest_ind(i_array, j_array)
                u_stat, u_p_value = mannwhitneyu(i_array, j_array)
                df.loc[len(df.index)] = [mutant, t_p_value, u_p_value]
            df.to_csv(f'analyzed-data/{choice}/{seed_i}_{seed_j}_{project}.csv', sep=',', header=True, index=False)
            j += 1

In [ ]:
# Parse T-test/U-test results
import pandas as pd
import numpy as np
from pitest_log_parser import project_list, round_number, seed_list, mutant_choice, test_choice
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'
seed_num = len(seed_list)

for project in project_list:
    p_value_record_df = pd.DataFrame(None, columns=['seed-pair', 'mutant', 'T-test', 'U-test'])
    for i in range(seed_num):
        seed_i = seed_list[i]
        j = i + 1
        while j < seed_num:
            seed_j = seed_list[j]
            pair = (seed_i, seed_j)
            total_cnt = 0
            significance_cnt = 0
            df = pd.read_csv(f'analyzed-data/{choice}/{seed_i}_{seed_j}_{project}.csv')
            for index, row in df.iterrows():
                total_cnt += 1
                t_test = float(row['T-test'])
                u_test = float(row['U-test'])
                if t_test < 0.05 or u_test < 0.05:
                    significance_cnt += 1
                    p_value_record_df.loc[len(p_value_record_df.index)] = [pair, row['mutant'], t_test, u_test]
            j += 1
            print(f'{project}-({seed_i}, {seed_j}): {significance_cnt}/{total_cnt}')
    p_value_record_df.to_csv(f'analyzed-data/{choice}/significance-record_{project}.csv', sep=',', header=True, index=False)        

In [3]:
# Find out falky test.
import pandas as pd
import numpy as np
from pitest_log_parser import project_list, round_number, seed_list, mutant_choice, test_choice
random_mutant = True
random_test = False
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'
seed_num = len(seed_list)

for project in project_list:
    for seed in seed_list:
        df = pd.read_csv(f'analyzed-data/{choice}/{seed}_{project}.csv')
        withoutTestSet_numberOfMutant_dict = {}
        withoutTestSet_testSet_dict = {}
        for index, row in df.iterrows():
            mutant = eval(row['mutant'])
            key = mutant[:-1]
            if random_test:
                if key in withoutTestSet_numberOfMutant_dict:
                    withoutTestSet_numberOfMutant_dict[key] += 1
                else:
                    withoutTestSet_numberOfMutant_dict[key] = 1
            else:
                test_set = tuple(set(mutant[-1]))
                if key in withoutTestSet_testSet_dict:
                    if test_set in withoutTestSet_testSet_dict[key]:
                        continue
                    withoutTestSet_testSet_dict[key].add(test_set)
                    withoutTestSet_numberOfMutant_dict[key] += 1
                else:
                    withoutTestSet_testSet_dict[key] = {test_set}
                    withoutTestSet_numberOfMutant_dict[key] = 1
        flakiness_df = pd.DataFrame(None, columns=['mutant-withoutTestSet', 'numberOfMutant'])
        for key, value in withoutTestSet_numberOfMutant_dict.items():
            flakiness_df.loc[len(flakiness_df.index)] = [key, value]
        flakiness_df_sorted = flakiness_df.sort_values(by='numberOfMutant', ascending=False)
        flakiness_df_sorted.to_csv(f'analyzed-data/{choice}/flakiness-record_{seed}_{project}', sep=',', header=True, index=False)